In [5]:
import os
from PIL import Image
import numpy as np
from skimage import measure

def process_mask_image(mask_image_path, output_txt_path, class_index):
    # Open the mask image and convert to grayscale
    mask_image = Image.open(mask_image_path).convert('L')
    width, height = mask_image.size

    # Convert the mask to a binary array
    mask_array = np.array(mask_image)

    # Threshold the mask to binary
    mask_array = (mask_array > 128).astype(np.uint8)

    # Find contours
    contours = measure.find_contours(mask_array, 0.5)

    with open(output_txt_path, 'w') as f:
        for contour in contours:
            if len(contour) < 6:
                # Ignore small contours that can't form a polygon
                continue

            # Flip (row, col) to (x, y)
            contour = np.flip(contour, axis=1)

            # Simplify the contour if necessary (optional)
            # Uncomment the following lines to simplify the contour
            # from shapely.geometry import Polygon
            # polygon = Polygon(contour)
            # polygon = polygon.simplify(0.005, preserve_topology=False)
            # xs, ys = polygon.exterior.coords.xy
            # contour = np.column_stack((xs, ys))

            # Normalize contour coordinates
            xs = contour[:, 0] / width
            ys = contour[:, 1] / height

            # Flatten the normalized coordinates
            contour_coords = np.column_stack((xs, ys)).ravel().tolist()

            # Build the line to write in the format:
            # <class-index> <x1> <y1> <x2> <y2> ... <xn> <yn>
            line = f"{class_index} " + ' '.join(map(str, contour_coords))
            f.write(line + '\n')

def process_mask_images(mask_images_dir, output_dir, class_index=0):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    mask_images = [f for f in os.listdir(mask_images_dir) if f.endswith('.png') or f.endswith('.jpg')]
    for mask_image_name in mask_images:
        mask_image_path = os.path.join(mask_images_dir, mask_image_name)
        output_txt_name = os.path.splitext(mask_image_name)[0] + '.txt'
        output_txt_path = os.path.join(output_dir, output_txt_name)
        process_mask_image(mask_image_path, output_txt_path, class_index)


for i in range(1, 24):
    # Directory containing the mask images
    mask_images_dir = f'./segmentation_data_new/test/labels/mask_{f"0{i}" if i < 10 else i}/'
    
    # Directory where the annotation .txt files will be saved
    output_dir = './segmentation_data_new/test/labels/'
    
    # Class index for the object
    class_index = i  # Adjust according to your class indexing
    
    process_mask_images(mask_images_dir, output_dir, class_index)
